# Кластеризация и снижение размерности



Мы будем работать с относительно простым датасетом по ирисам Фишера. Загрузить данные можно кодом из ячейки ниже прямо из библиотеки sklearn. 

\* Для желающих попробовать датасет посложнее, рекомендую скачать с Kaggle [данные](https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset) по пациентам с сердечно-сосудистыми заболеваниями. Предобработку выполните самостоятельно. Можно опираться на следующий [ноутбук](https://www.kaggle.com/code/sulianova/eda-cardiovascular-data/notebook)

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris_df = load_iris(as_frame=True)["frame"]
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
species_names = load_iris()["target_names"]
species_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [4]:
iris_df["target"] = species_names[iris_df["target"]]
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Эти данные содержат информацию по длине и ширине чашелистиков (sepal) и листков (petal) ирисов 3 видов. Ваша задача – исследовать эти данные и посмотреть как на них себя ведут разные алгоритмы кластеризации. Звёздочками обозначены не обязательные задачи. Использовать столбец "target" для кластеризации и снижения размерности **не нужно**. Мы будем сравнивать, способны ли алгоритмы воспроизвести "человеческую" классификацию цветов.


1. **Подумайте, нужно ли преобразовывать числовые данные? Проверьте, имеют ли они одинаковый масштаб, и примите решение о предобработке или обоснуйте, почему можно этого не делать**

    \* Создайте датафреймы с предобработкой и без, сравните результаты в последующих заданиях

2. **Кластеризуйте данные при помощи разных алгоритмов:**
- K-means (`sklearn.cluster.KMeans`). Используйте количество кластеров равное 3, так как мы предполагаем 3 вида в данных

    - \* Запустите алгоритм с разным значением K, посчитайте silhouette score (`sklearn.metrics.silhouette_score`) и определите оптимальное количество кластеров

- Иерархическая кластеризация (`sklearn.cluster.AgglomerativeClustering`)
- DBSCAN (`sklearn.cluster.DBSCAN`) 
    - Попробуйте разные значения параметров `eps` (минимальное расстояние) и `min_samples` (минимальный размер кластера). Какое количетсво кластеров получается в разных случаях? Есть ли наблюдения, отмеченные как шум (кластер `-1`)?
- [Leiden](https://leidenalg.readthedocs.io/en/stable/intro.html)

3. **Сравните результат кластеризации разными методами.**
- Насколько кластеры от разных методов пересекаются?
- Насколько кластеры похожи на настоящие классы (виды ирисов из колонки `target` или наличие заболевания из колонки `cardio`)?
- Вам может пригодиться функция `pd.crosstab`

4. **Интерпретируйте результат кластеризации**. Возможные способы (попробуйте все или выберите то, что нравится):
- Посмотрите на средние значения признаков по кластерам
- Визуализируйте кластеры на точечных графиках по подходящим признакам
- Постройте распределения признаков по кластерам
- \* Запустите подходящие статистические тесты

Для примеров кода по алгоритмам кластеризации воспользуйтесь документацией [sklearn](https://scikit-learn.org/stable/modules/clustering.html#)

Для алгоритма leiden необходимо построить граф расстояний и пересчитать их в "похожести". На мой взгляд, удобнее всего сделать это через библиотеку для анализа данных транскриптомики scanpy. Установите её при помощи `pip install scanpy` и пользуйтесь следующей функцией:

In [5]:
def get_leiden_clusters(data, n_neighbors: int = 15, resolution: float = 1.0):
    """Perform leiden clustering by building neighbors graph from numerical data
    
    Parameters
    ----------
    data : matrix or data frame
        Your data where rows are observations and columns are features. IMPORTANT:
        features must be numerical
    n_neighbors : int = 15
        Number of neighbors in KNN-graph
    resolution : float = 1.0
        Resolution parameter for leiden algorithm
        
    Returns
    -------
    clusters : pd.Series
        Series with clusters for your data
    """
    import scanpy as sc
    
    adata = sc.AnnData(data)  # Convert data to annotated data object
    sc.pp.neighbors(adata, n_neighbors=n_neighbors)  # Build neighbors graph
    sc.tl.leiden(adata, resolution=resolution)  # Run clustering
    return adata.obs["leiden"].to_numpy()

5. **Выполните понижение размерности при помощи метода PCA** (`sklearn.decomposition.PCA`)
- Визуализируйте процент объяснённой дисперсии
- Визуализируйте первые 2 компоненты
- При помощи анализа коэффициентов признаков (loadings) или визуально по графикам интерпретируйте, что означают главные компоненты для этих данных
- \* Изменяется ли результат кластеризации, если вместо исходных признаков использовать только несколько главных компонент? Сравните результат любого метода на выбор или всех

6. **Выполните визуализацию данных при помощи метода UMAP (библиотека [umap-learn](https://umap-learn.readthedocs.io/en/latest/)**
- Как на графике UMAP представлены виды цветов (или наличие заболевания для датасета по пациентам)? А отдельные признаки?
- Изменяется ли ваше понимание этих данных от этого вида визуализации? Удаётся ли обнаружить что-то необычное?